In [ ]:
!pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-non

In [ ]:
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
data = load_dataset('csv', data_files={'train': "gdrive/My Drive/recipe_dataset_10000.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = data["train"].train_test_split(test_size=0.2)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 3200
    })
    test: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 800
    })
})

In [ ]:
dataset['train'][0]

{'input_text': 'How to make Chili Colorado?',
 'target_text': "You can make Chili Colorado with 8 dried new mexico peppers, 6 dried arbol chiles, 6 dried ancho chiles, 4 dried chipotle chiles, 3 cups boiling water, 4 lbs sirloin, in 1 or 2 large pieces, 8 slices bacon, 1 teaspoon Mexican oregano or 1 1/2 tablespoons oregano, 1 1/2 teaspoons cumin, 1 teaspoon salt, 1 large onion, 5 -6 cloves garlic, 2 cups clear beef broth, 1/2 cup chopped cilantro. Here's the instruction: Take all the chiles and seed. Snip with sissors and place in a medium glass bowl. Cover with boiling water and then cover bowl with a plate. Let stand for 30 minutes. Mean while brown bacon in a large skillet. Remove and add sirloin steak to the skillet and brown in bacon fat on both sides. Add the meats to the crock pot. Place the chiles and water in a food processor and whirl to very smooth. Strain if you want, I don't worry about it when using the processor. by hand is another story. You will need to press puree th

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("sjrhuschlee/flan-t5-base-squad2", local_files_only=True)
model = T5ForConditionalGeneration.from_pretrained("sjrhuschlee/flan-t5-base-squad2", local_files_only=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:


# Define our preprocessing function
def preprocess_function(examples):
    # The "inputs" are the tokenized answer:
    model_inputs = tokenizer(examples["input_text"], max_length=512, padding='max_length', truncation=True)

    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["target_text"], max_length=512, padding='max_length', truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Map the preprocessing function across our dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

In [ ]:
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    push_to_hub=False
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,0.893085,0.164373,0.101263,0.151069,0.159249
2,1.010300,0.873403,0.164742,0.100838,0.151411,0.159738
3,0.931600,0.867886,0.165100,0.100943,0.151806,0.160029


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=1200, training_loss=0.9631536483764649, metrics={'train_runtime': 1107.0078, 'train_samples_per_second': 8.672, 'train_steps_per_second': 1.084, 'total_flos': 6573670583500800.0, 'train_loss': 0.9631536483764649, 'epoch': 3.0})

In [ ]:
#trainer.evaluate()

In [ ]:
trainer.save_model ("gdrive/My Drive/T5_10k2")